In [1]:
import numpy as np
import os
import pickle
import copy 
import json
import warnings
warnings.filterwarnings("ignore")
from load_img import Load_from_Folder, Load_Images
from evaluate import Time, MSE, PSNR
from LANCZOS import LANCZOS
from util import DownSample
from GIC import GIC

def run(par, train_path, n_train, test_path, n_test, ratio=4):
    print('--- <START TRAIN> ---')
    Y = Load_from_Folder(folder=train_path, color='YUV', ct=n_train)
    Y = np.array(Y)
    Y = DownSample(Y, ratio)
    gic = GIC(par)
    gic.fit(Y)
   
    print('--- <START TEST> ---')
    Y = Load_from_Folder(folder=test_path, color='YUV', ct=10)
    Y = np.array(Y)
    Y = DownSample(Y, ratio)
    stream, dc, save = gic.encode(Y)
    iY = gic.decode(stream, dc)
    print('MSE=%5.3f, PSNR=%3.5f'%(MSE(Y, iY), PSNR(Y, iY)))
    return gic

In [2]:
with open('par_unit_test.json', 'r') as json_file:
    par = json.load(json_file)
print(par)

{'n_hop': 6, 'win': {'hop1': [256, 128, 64, 32, 16, 8], 'hop2': [128, 64, 32, 16, 8, 4], 'hop3': [64, 32, 16, 8, 4], 'hop4': [32, 16, 8, 4], 'hop5': [16, 8, 4], 'hop6': [8, 4]}, 'MSE_TH': {'hop1': [-1, 0, 0, 10, 20, 30, 40], 'hop2': [-1, 0, 10, 20, 30, 40], 'hop3': [-1, 0, 0, 0, 0], 'hop4': [-1, 0, 0, 0], 'hop5': [-1, 0, 0], 'hop6': [-1, 0]}, 'n_clusters': {'hop1': [4, 4, 4, 4, 4, 4, 4], 'hop2': [4, 4, 4, 4, 4, 4], 'hop3': [4, 4, 4, 4, 4], 'hop4': [4, 4, 4, 4], 'hop5': [4, 4, 4], 'hop6': [4, 4]}, 'shape': {'hop1': [[-1, 1, 1, 1], [-1, 2, 2, 1], [-1, 4, 4, 1], [-1, 8, 8, 1], [-1, 16, 16, 1], [-1, 32, 32, 1]], 'hop2': [[-1, 1, 1, 1], [-1, 2, 2, 1], [-1, 4, 4, 1], [-1, 8, 8, 1], [-1, 16, 16, 1], [-1, 32, 32, 1]], 'hop3': [[-1, 1, 1, 1], [-1, 2, 2, 1], [-1, 4, 4, 1], [-1, 8, 8, 1], [-1, 16, 16, 1]], 'hop4': [[-1, 1, 1, 1], [-1, 2, 2, 1], [-1, 4, 4, 1], [-1, 8, 8, 1]], 'hop5': [[-1, 1, 1, 1], [-1, 2, 2, 1], [-1, 4, 4, 1]], 'hop6': [[-1, 1, 1, 1], [-1, 2, 2, 1]]}, 'n_components': {'hop1': [1

In [10]:
gic = run(par, 
          '/Users/alex/Desktop/proj/compression/data/train1k/', 8,
          '/Users/alex/Desktop/proj/compression/data/test_1024/', 1)

--- <START TRAIN> ---
   <RunTime> foreward:  0.2 s
hop6
MBVQ channel 3
   <RunTime> backward:  1.3 s
hop5
MBVQ channel 3
   <RunTime> backward:  2.6 s
hop4
MBVQ channel 3
   <RunTime> backward:  4.1 s
hop3
MBVQ channel 3
   <RunTime> backward:  7.1 s
hop2
MBVQ channel 3
   <RunTime> backward: 13.2 s
hop1
MBVQ channel 3

Training MSE: 30.212790997438034
   <RunTime> foreward:  0.1 s
hop6
   <RunTime> backward:  0.0 s
hop5
   <RunTime> backward:  0.0 s
hop4
   <RunTime> backward:  0.0 s
hop3
   <RunTime> backward:  0.1 s
hop2
   <RunTime> backward:  0.3 s
hop1

Testing MSE: 30.212790997438034
   <RunTime> fit: 82.8 s
--- <START TEST> ---
   <RunTime> foreward:  0.2 s
hop6
   <RunTime> backward:  0.0 s
hop5
   <RunTime> backward:  0.0 s
hop4
   <RunTime> backward:  0.0 s
hop3
   <RunTime> backward:  0.1 s
hop2
   <RunTime> backward:  0.4 s
hop1

Testing MSE: 835.5992747119213
Hop-1
   save tree:0.00519104
  Level-0 Huffman 0.00002899 bpp
  Level-1 Huffman 0.00006104 bpp
  Level-2 Huffman

In [21]:
for k in gic.MBMBVQ.km.keys():
    km = gic.MBMBVQ.km[k]
    for i in km.KM:
        i.KM.KM = None
        i.KM.saveObj=False
with open('model.pkl', 'wb') as f:
    pickle.dump(gic, f)

In [22]:
with open('model.pkl', 'rb') as f:
    gic = pickle.load(f)
    
Y = Load_from_Folder(folder='/Users/alex/Desktop/proj/compression/data/test_1024/', color='YUV', ct=10)
Y = np.array(Y)
Y = DownSample(Y, 4)
stream, dc, save = gic.encode(Y)
iY = gic.decode(stream, dc)
print('MSE=%5.3f, PSNR=%3.5f'%(MSE(Y, iY), PSNR(Y, iY)))

   <RunTime> foreward:  0.2 s
hop6
   <RunTime> backward:  0.0 s
hop5
   <RunTime> backward:  0.0 s
hop4
   <RunTime> backward:  0.0 s
hop3
   <RunTime> backward:  0.1 s
hop2
   <RunTime> backward:  0.4 s
hop1

Testing MSE: 835.5992747119213
Hop-1
   save tree:0.00519104
  Level-0 Huffman 0.00002899 bpp
  Level-1 Huffman 0.00006104 bpp
  Level-2 Huffman 0.00027313 bpp
  Level-3 Huffman 0.00098877 bpp
  Level-4 Huffman 0.00661774 bpp
  Level-5 Huffman 0.02784424 bpp
Hop-2
   save tree:0.00509338
  Level-0 Huffman 0.00001526 bpp
  Level-1 Huffman 0.00006104 bpp
  Level-2 Huffman 0.00032349 bpp
  Level-3 Huffman 0.00178528 bpp
  Level-4 Huffman 0.00729370 bpp
  Level-5 Huffman 0.02874756 bpp
Hop-3
   save tree:0.00129700
  Level-0 Huffman 0.00001526 bpp
  Level-1 Huffman 0.00006104 bpp
  Level-2 Huffman 0.00040131 bpp
  Level-3 Huffman 0.00150146 bpp
  Level-4 Huffman 0.00781250 bpp
Hop-4
   save tree:0.00032043
  Level-0 Huffman 0.00001526 bpp
  Level-1 Huffman 0.00006104 bpp
  Level-2 H